In [6]:
import os
import re
import sys
import math
import json
import requests
import pandas as pd
from glob import glob
from tqdm import tqdm

# ===== 모듈 경로 추가 & 로거 설정 =====
sys.path.append(r"C:\ESG_Project1\util")
from logger import setup_logger
logger = setup_logger(__name__)

# ----------------------------------------------------
# 경로 설정
# ----------------------------------------------------
BASE_DIR = "C:/ESG_Project1/file/"
SOLAR_DIR = os.path.join(BASE_DIR, "solar_data_file")
SOLAR_LOCATION_DIR = os.path.join(BASE_DIR, "solar_data_file/location")
KMA_DIR = os.path.join(BASE_DIR, "KMA_data_file")
WEATHER_META = os.path.join(KMA_DIR, "META_관측지점정보.csv")
MANAGED_JSON = os.path.join(SOLAR_LOCATION_DIR, "json/managed_human.json")
LOCATION_JSON = os.path.join(SOLAR_LOCATION_DIR, "json/cache_location.json")
OUT_CSV = os.path.join(BASE_DIR, "merge_data/test_data2.csv")

TARGET_YEARS = list(range(2024, 2026))
KAKAO_API_KEY = "93c089f75a2730af2f15c01838e892d3"

# ----------------------------------------------------
# CSV 유틸
# ----------------------------------------------------
def sniff_delimiter(path):
    with open(path,"rb") as f:
        raw=f.read(2048)
    text=raw.decode("utf-8",errors="ignore")
    return "," if text.count(",") >= text.count("\t") else "\t"

def read_csv_safe(path):
    delim = sniff_delimiter(path)
    try:
        df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
        logger.info(f"📄 '{os.path.basename(path)}' 읽기 성공 (utf-8)")
        return df
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="cp949", delimiter=delim, index_col=False)
        logger.info(f"📄 '{os.path.basename(path)}' 읽기 성공 (cp949)")
        return df

# ----------------------------------------------------
# 문자열 정규화 & 컬럼 탐지
# ----------------------------------------------------
def norm(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[\s\-\_\/\(\)\[\]\.]+", "", s)
    return s

PLANT_TOKENS = ["발전소명","발전소","호기명","호기","플랜트","사업명","사업장","설비명","site","plant","facility","name"]
LOC_TOKENS   = ["위치","주소","지역","시도","시군구","행정구역","소재지","address","location","region","city","province","lat","lon","latitude","longitude"]
SRC_TOKENS   = ["에너지원","에너지","발전원","전원","연료","source","fuel","type"]

def detect_col(columns, tokens):
    for c in columns:
        c_norm = norm(c)
        for t in tokens:
            if norm(t) in c_norm:
                return c
    return None

# ----------------------------------------------------
# 주소 단순화
# ----------------------------------------------------
def simplify_address(addr: str) -> str:
    if not isinstance(addr, str): return addr
    addr = addr.strip()
    if not addr: return addr
    short_province_pattern = r"^(전남|전북|경남|경북|충남|충북|강원|제주|서울|부산|대구|인천|광주|대전|울산|세종)"
    long_province_pattern = r"^(서울특별시|부산광역시|대구광역시|인천광역시|광주광역시|대전광역시|울산광역시|세종특별자치시|경기도|강원도|강원특별자치도|충청북도|충청남도|전라북도|전라남도|경상북도|경상남도|제주특별자치도)"
    tokens = []
    parts = re.split(r"\s+", addr)
    if re.match(long_province_pattern, parts[0]): tokens.append(parts[0]); parts = parts[1:]
    elif re.match(short_province_pattern, parts[0]): tokens.append(parts[0]); parts = parts[1:]
    for p in parts:
        if re.search(r"(시|군|구|읍|면|동)$", p): tokens.append(p)
        if len(tokens) >= 3: break
    if not tokens: return " ".join(addr.split()[:3])
    return " ".join(tokens)

# ----------------------------------------------------
# Haversine 거리 계산
# ----------------------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# ----------------------------------------------------
# 카카오 API + 캐시
# ----------------------------------------------------
CACHE_PATH = os.path.join(SOLAR_LOCATION_DIR, "json/cache_latlon.json")
if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, "r", encoding="utf-8") as f: CACHE = json.load(f)
else: CACHE = {}

def get_latlon_from_kakao(address: str):
    if not isinstance(address, str) or not address.strip(): return None, None
    addr_key = address.strip()
    if addr_key in CACHE: return CACHE[addr_key]["lat"], CACHE[addr_key]["lon"]
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": addr_key}
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=5)
        data = resp.json()
        if data.get("documents"):
            doc = data["documents"][0]
            lat, lon = float(doc["y"]), float(doc["x"])
            CACHE[addr_key] = {"lat": lat, "lon": lon}
            with open(CACHE_PATH, "w", encoding="utf-8") as f: json.dump(CACHE, f, ensure_ascii=False, indent=2)
            return lat, lon
    except Exception as e:
        logger.info(f"⚠️ 카카오 API 실패: {address} → {e}")
    return None, None

# ----------------------------------------------------
# CSV 처리 + 발전량 변환 함수
# ----------------------------------------------------
def extract_hour_from_col(colname):
    s = str(colname).strip().lower()
    m = re.search(r"(\d{1,2})\s*[-~_]\s*(\d{1,2})", s)
    if m: return int(m.group(2)) if 1 <= int(m.group(2)) <= 24 else None
    m = re.search(r"(\d{1,2})\s*시", s)
    if m: return int(m.group(1)) if 1 <= int(m.group(1)) <= 24 else None
    m = re.search(r"(^|\D)(\d{1,2})(?!\d)", s)
    if m: return int(m.group(2)) if 1 <= int(m.group(2)) <= 24 else None
    return None

def detect_hour_cols(df):
    candidates = []
    for c in df.columns:
        h = extract_hour_from_col(c)
        if h is not None: candidates.append((c, h))
    return [c for c, h in sorted(candidates, key=lambda x:x[1])]

def detect_unit(colname):
    name = str(colname).lower()
    if "mwh" in name: return "mwh"
    elif "kwh" in name: return "kwh"
    elif "wh" in name: return "wh"
    return None

def detect_unit_from_cols(cols):
    for c in cols:
        u = detect_unit(c)
        if u: return u
    return None

def convert_to_mwh(val, unit):
    try:
        val = float(val)
        if unit=="wh": val/=1_000_000
        elif unit=="kwh": val/=1_000
        return round(val,4)
    except: return 0.0

def extract_plant_name(filename):
    match = re.search(r"\[(.*?)\]", filename)
    return match.group(1).strip() if match else None

# ----------------------------------------------------
# 발전량 + 위치 매핑 + KMA 병합
# ----------------------------------------------------
all_frames = []

files = sorted(glob(os.path.join(SOLAR_DIR, "**", "*.csv"), recursive=True))
for f in tqdm(files, desc="CSV 처리 중"):
    try:
        df = read_csv_safe(f)
        name_col = detect_col(df.columns, PLANT_TOKENS)
        date_col = detect_col(df.columns, ["일자","날짜","년월일","date"])
        ho_col   = detect_col(df.columns, ["호기","unit"])
        hour_cols = detect_hour_cols(df)
        plant_name = extract_plant_name(os.path.basename(f))

        if not hour_cols or not date_col: continue

        if name_col: df["발전소명"] = df[name_col].astype(str).str.strip()
        else: df["발전소명"] = plant_name if plant_name else "미상"

        unit = detect_unit_from_cols(hour_cols)
        if not unit: unit="kwh"

        df_long = df.melt(id_vars=[c for c in [date_col, ho_col, "발전소명"] if c in df.columns],
                          value_vars=hour_cols, var_name="시간대", value_name="발전량(MWh)")
        df_long[date_col] = pd.to_datetime(df_long[date_col].astype(str).str.replace(r"[^0-9]","",regex=True).str[:8], format="%Y%m%d", errors="coerce")
        df_long = df_long[df_long[date_col].dt.year.isin(TARGET_YEARS)]
        hours = df_long["시간대"].apply(extract_hour_from_col).astype("Int64")
        df_long = df_long[hours.notna()].copy()
        df_long["일시"] = df_long[date_col] + pd.to_timedelta(hours-1, unit="h")
        df_long["발전량(MWh)"] = df_long["발전량(MWh)"].apply(lambda x: convert_to_mwh(x, unit))
        df_clean = df_long[["일시","발전소명","발전량(MWh)"]].copy()
        all_frames.append(df_clean)
    except Exception as e:
        logger.error(f"❌ 오류: {os.path.basename(f)} → {e}")

if not all_frames: raise SystemExit("❌ 처리할 데이터 없음")

merged = pd.concat(all_frames, ignore_index=True)
final_df = merged.groupby(["발전소명","일시"], as_index=False)["발전량(MWh)"].sum()
final_df["일시"] = final_df["일시"].dt.strftime("%Y-%m-%d %H:%M:%S")
final_df["발전량(MWh)"] = final_df["발전량(MWh)"].apply(lambda x: f"{x:.4f}")

# ----------------------------------------------------
# location + KMA 병합
# ----------------------------------------------------
try:
    with open(MANAGED_JSON, "r", encoding="utf-8") as f: managed_map = json.load(f)
except: managed_map = {}
final_df["매핑용발전소명"] = final_df["발전소명"].map(managed_map).fillna(final_df["발전소명"])

try:
    with open(LOCATION_JSON,"r",encoding="utf-8") as f: loc_df=pd.DataFrame(json.load(f))
except: loc_df=pd.DataFrame(columns=["발전소명","지점","지점명"])
loc_df = loc_df[["발전소명","지점","지점명"]].drop_duplicates(subset=["발전소명"])

final_df = pd.merge(final_df, loc_df, left_on="매핑용발전소명", right_on="발전소명", how="left", suffixes=("","_loc"))
if "지점" in final_df.columns: final_df["지점"] = pd.to_numeric(final_df["지점"], errors="coerce").astype("Int64")
final_df.drop(columns=["매핑용발전소명","발전소명_loc"], inplace=True, errors="ignore")
final_df = final_df[["일시","발전소명","지점명","지점","발전량(MWh)"]]

# KMA 데이터 병합
weather_files = sorted(glob(os.path.join(KMA_DIR,"OBS_ASOS_TIM_*.csv")))
weather_frames = []
for wf in tqdm(weather_files, desc="기상데이터 병합"):
    try:
        tmp = read_csv_safe(wf)
        tmp["일시"] = pd.to_datetime(tmp["일시"], errors="coerce", format="mixed")
        tmp = tmp[["지점","지점명","일시","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]]
        weather_frames.append(tmp)
    except Exception as e:
        logger.warning(f"⚠️ 기상파일 '{os.path.basename(wf)}' 처리 실패: {e}")
df_weather = pd.concat(weather_frames, ignore_index=True).drop_duplicates(subset=["지점","일시"])

final_df["일시"] = pd.to_datetime(final_df["일시"], errors="coerce").dt.floor("h")
df_weather["일시"] = pd.to_datetime(df_weather["일시"], errors="coerce").dt.floor("h")
final_df = pd.merge(final_df, df_weather, how="left", on=["지점","일시"])
final_df[["발전량(MWh)","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]] = \
    final_df[["발전량(MWh)","기온(°C)","강수량(mm)","일조(hr)","일사(MJ/m2)"]].fillna(0)

# 컬럼 정리
final_df = final_df.rename(columns={
    "발전소명":"발전구분",
    "지점명":"지역",
    "지점":"지점번호",
    "발전량(MWh)":"합산발전량(MWh)"
})
final_df = final_df.sort_values(by=["발전구분","일시"]).reset_index(drop=True)

# 저장
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
final_df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 발전소-지점 매핑 + 기상데이터 병합 완료 → {OUT_CSV}")
logger.info(f"✅ 발전소-지점 매핑 + 기상데이터 병합 완료 → {OUT_CSV} (총 {len(final_df):,}행)")


CSV 처리 중: 

[2025-10-27 12:00:32,891]✅ INFO - 📄 '한국남동발전㈜_(발전공기업 표준) 신재생에너지 사업현황_20221231.csv' 읽기 성공 (cp949)
[2025-10-27 12:00:32,893]✅ INFO - 📄 '한국남부발전(주)_태양광발전기 사양정보_20250630.csv' 읽기 성공 (cp949)
[2025-10-27 12:00:32,895]✅ INFO - 📄 '한국동서발전(주)_(발전공기업 표준)신재생에너지 사업현황_20230630.csv' 읽기 성공 (cp949)
[2025-10-27 12:00:32,897]✅ INFO - 📄 '한국서부발전(주)_신재생에너지사업현황_20231231.csv' 읽기 성공 (cp949)
[2025-10-27 12:00:32,900]✅ INFO - 📄 '남동발전량_2022_01.csv' 읽기 성공 (utf-8)
[2025-10-27 12:00:32,924]✅ INFO - 📄 '남동발전량_2022_02.csv' 읽기 성공 (utf-8)
[2025-10-27 12:00:32,945]✅ INFO - 📄 '남동발전량_2022_03.csv' 읽기 성공 (utf-8)
[2025-10-27 12:00:32,968]✅ INFO - 📄 '남동발전량_2022_04.csv' 읽기 성공 (utf-8)
[2025-10-27 12:00:32,990]✅ INFO - 📄 '남동발전량_2022_05.csv' 읽기 성공 (utf-8)


C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, 

[2025-10-27 12:00:33,016]✅ INFO - 📄 '남동발전량_2022_06.csv' 읽기 성공 (utf-8)
[2025-10-27 12:00:33,039]✅ INFO - 📄 '남동발전량_2022_07.csv' 읽기 성공 (utf-8)
[2025-10-27 12:00:33,061]✅ INFO - 📄 '남동발전량_2022_08.csv' 읽기 성공 (utf-8)
[2025-10-27 12:00:33,084]✅ INFO - 📄 '남동발전량_2022_09.csv' 읽기 성공 (utf-8)


C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\c\AppData\Local\Temp\ipykernel_24888\913769041.py:43: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path, 

...

[2025-10-27 12:00:45,843]✅ INFO - 📄 'OBS_ASOS_TIM_2024.csv' 읽기 성공 (cp949)


기상데이터 병

[2025-10-27 12:00:46,352]✅ INFO - 📄 'OBS_ASOS_TIM_2025.csv' 읽기 성공 (cp949)


기상데이터 병


✅ 발전소-지점 매핑 + 기상데이터 병합 완료 → C:/ESG_Project1/file/merge_data/test_data2.csv
[2025-10-27 12:00:49,481]✅ INFO - ✅ 발전소-지점 매핑 + 기상데이터 병합 완료 → C:/ESG_Project1/file/merge_data/test_data2.csv (총 119,136행)
